In [ ]:
ntraj = 24
ihmm = [[] for i in range(ntraj)]

In [ ]:

res = 'MET'
params = file_rw.load_object('saved_parameters/final_parameters_agglomerative_MET_eigs_combined_30.pl')

nsteps = 4808

trajectory_generator = GenARData(params=params)
trajectory_generator.gen_trajectory(nsteps, ntraj, bound_dimensions=[0], resample_T=False)

file_rw.save_object((trajectory_generator.traj, 0.5), 'trajectories/toy_data_%s.pl' % res)

In [ ]:
# initialize the ihmm for each trajectory

res='MET'
difference = False  # take first order difference of solute trajectories
observation_model='AR'  # assume an autoregressive model (that's the only model implemented)
order = 1  # autoregressive order
max_states = 100  # More is usually better
traj_no = np.arange(24).tolist() # np.arange(10).tolist()# [2]# None # np.arange(24)#2
dim = [0, 1, 2]  # dimensions of trajectory to keep
prior = 'MNIW-N'  # MNIW-N (includes means) or MNIW (forces means to zero)
save_every = 1
#com = 'trajectories/toy_data.pl' # saved center of mass trajectories
com = 'trajectories/toy_data_%s.pl' % res
load_com=True
#hyperparams={'scale_sig0': np.array([1, 1]), 'a_gamma': 50}
hyperparams=None

# for t in traj_no:
#     print('Trajectory %d' % t)
    
#     ihmm[t] = hdphmm.InfiniteHMM(com, traj_no=t, load_com=load_com, difference=difference, 
#                              observation_model=observation_model, order=order, max_states=max_states,
#                              dim=dim, prior=prior, save_every=save_every, hyperparams=hyperparams)

In [ ]:
#com = 'trajectories/toy_data_%s.pl' % res
# com_raw = file_rw.load_object('trajectories/toy_data_%s.pl' % res)
#com_raw = file_rw.load_object('trajectories/toy_data.pl')
res = 'MET'
seed_seq = True
traj_no = np.arange(24)
com_filename = 'trajectories/com_xy_radial_%s.pl' % res
seed_seq = True

if seed_seq:
    com = file_rw.load_object(com_filename)
else:
    com = com_filename

for t in traj_no:
    
    if seed_seq:
        z = seed_sequence(com_raw, t, nseg=4, max_states=max_states, niter=5)
        print('Seeding with %d states' % np.unique(z).size)
        com = (com[0][:(z.size + 1), ...], com[1])
        load_com = False
    else:
        
        z = None
        load_com = True

    ihmm[t] = hdphmm.InfiniteHMM(com, traj_no=[t], load_com=load_com, difference=difference, 
                             observation_model=observation_model, order=order, max_states=max_states,
                             dim=dim, prior=prior, save_every=save_every, hyperparams=hyperparams,
                             seed_sequence=z)

In [ ]:
niter = 100
for i in traj_no:
    ihmm[i].inference(niter)
    
for i in traj_no:
    ihmm[i]._get_params(quiet=True)

In [ ]:
for t in traj_no:
    ihmm[t].summarize_results(savename='/home/ben/github/LLC_Membranes/Ben_Manuscripts/hdphmm/supporting_figures/many_states.pdf')

In [ ]:
ntrajper=10
endshow = 2000
MD_MSD = file_rw.load_object('trajectories/%s_msd.pl' % res)
time = np.arange(endshow) * 0.5
confidence = 68

unclustered_trajectory_realizations = np.zeros([len(ihmm), ihmm[traj_no[0]].z.shape[1], ntrajper, ihmm[traj_no[0]].dimensions])

for t in traj_no:
    
    final_p = get_params([ihmm[t]], ihmm[t], ihmm[t].z, clustered=False)
#     if mod_T:

#         final_p['T'] = modify_T(final_p['T'].mean(axis=0), percentile=20, increase_factor=2.25, decrease_factor=1)

    trajectory_generator = GenARData(params=final_p)

    for tr in tqdm.tqdm(range(ntrajper)):

        trajectory_generator.gen_trajectory(ihmm[t].z.shape[1], 1, bound_dimensions=[0, 1], resample_T=True, progress=False)

        unclustered_trajectory_realizations[i, :, tr, :] = trajectory_generator.traj[:, 0, :]

    plt.plot(time, MD_MSD.MSD[:endshow, t], color='black')

    msd = ts.msd(unclustered_trajectory_realizations[t, ...], 2, progress=False)
    msd_mean = msd.mean(axis=1)

    lower_confidence = (100 - confidence) / 2
    upper_confidence = 100 - lower_confidence

    error = np.zeros([2, msd.shape[0]], dtype=float)  # upper and lower bounds at each point along MSD curve
    # determine error bound for each tau (out of n MSD's, use that for the error bars)
    for s in range(error.shape[1]):
        error[0, s] = np.abs(np.percentile(msd[s, :], lower_confidence) - msd_mean[s])
        error[1, s] = np.abs(np.percentile(msd[s, :], upper_confidence) - msd_mean[s])

    plt.plot(time, msd.mean(axis=1)[:endshow], lw=2, color='xkcd:orange', label='IHMM')
    plt.fill_between(time, msd.mean(axis=1)[:endshow] + error[0, :endshow], msd.mean(axis=1)[:endshow] - error[1, :endshow], alpha=0.3, color='xkcd:orange')

    plt.show()

In [ ]:
for s in range(nseg):
    segments[s].summarize_results(traj_no=0)

In [ ]:
for i in traj_no:
    print('Trajectory %d' % i)
    ihmm[i].summarize_results(traj_no=0)

In [ ]:
mean_zero = []
dwells = []
hops = []

for s in range(nseg):
    
    zeroed, d, h = segments[s].subtract_mean(traj_no=0, simple_mean=True, return_dwells_hops=True)
    mean_zero.append(zeroed)
    dwells += d
    hops += h
    
hops = np.array(hops)[:, 1]
    
mean_zero = np.array(mean_zero)
mean_zero = mean_zero.reshape(mean_zero.shape[0] * mean_zero.shape[1], 2)
print(mean_zero.shape)

fig, ax = plt.subplots(2, 1, figsize=(12, 5), sharex=True)

ma_z = ts.calculate_moving_average(mean_zero[:, 1], 50)
ma_r = ts.calculate_moving_average(mean_zero[:, 0], 50)
print('z: %.3f' % ma_z.std())
print('r: %.3f' % ma_r.std())

ax[0].plot(ma_r)
ax[1].plot(ma_z)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from LLC_Membranes.llclib import timeseries as ts

mean_zero = []
dwells = []
hops = []

for t in traj_no:
    
    zeroed, d, h = ihmm[t].subtract_mean(traj_no=0, simple_mean=True, return_dwells_hops=True)
    mean_zero.append(zeroed)
    dwells += d
    hops += h
    
hops = np.array(hops)[:, 1]
    
mean_zero = np.array(mean_zero)

fig, ax = plt.subplots(2, 1, figsize=(12, 5), sharex=True)

ma_z = ts.calculate_moving_average(mean_zero[0, :, 1], 50)
ma_r = ts.calculate_moving_average(mean_zero[0, :, 0], 50)
print('z: %.3f' % ma_z.std())
print('r: %.3f' % ma_r.std())

ax[0].plot(ma_r)
ax[1].plot(ma_z)

plt.show()

In [ ]:
ntraj = 24
ihmm = [[] for i in range(ntraj)]

In [ ]:

res = 'MET'
params = file_rw.load_object('saved_parameters/final_parameters_agglomerative_MET_eigs_combined_30.pl')

nsteps = 4808

trajectory_generator = GenARData(params=params)
trajectory_generator.gen_trajectory(nsteps, ntraj, bound_dimensions=[0], resample_T=False)

file_rw.save_object((trajectory_generator.traj, 0.5), 'trajectories/toy_data_%s.pl' % res)

In [ ]:
# initialize the ihmm for each trajectory

res='MET'
difference = False  # take first order difference of solute trajectories
observation_model='AR'  # assume an autoregressive model (that's the only model implemented)
order = 1  # autoregressive order
max_states = 100  # More is usually better
traj_no = np.arange(24).tolist() # np.arange(10).tolist()# [2]# None # np.arange(24)#2
dim = [0, 1, 2]  # dimensions of trajectory to keep
prior = 'MNIW-N'  # MNIW-N (includes means) or MNIW (forces means to zero)
save_every = 1
#com = 'trajectories/toy_data.pl' # saved center of mass trajectories
com = 'trajectories/toy_data_%s.pl' % res
load_com=True
#hyperparams={'scale_sig0': np.array([1, 1]), 'a_gamma': 50}
hyperparams=None

# for t in traj_no:
#     print('Trajectory %d' % t)
    
#     ihmm[t] = hdphmm.InfiniteHMM(com, traj_no=t, load_com=load_com, difference=difference, 
#                              observation_model=observation_model, order=order, max_states=max_states,
#                              dim=dim, prior=prior, save_every=save_every, hyperparams=hyperparams)

In [ ]:
#com = 'trajectories/toy_data_%s.pl' % res
# com_raw = file_rw.load_object('trajectories/toy_data_%s.pl' % res)
#com_raw = file_rw.load_object('trajectories/toy_data.pl')
res = 'MET'
seed_seq = True
traj_no = np.arange(24)
com_filename = 'trajectories/com_xy_radial_%s.pl' % res
seed_seq = True

if seed_seq:
    com = file_rw.load_object(com_filename)
else:
    com = com_filename

for t in traj_no:
    
    if seed_seq:
        z = seed_sequence(com_raw, t, nseg=4, max_states=max_states, niter=5)
        print('Seeding with %d states' % np.unique(z).size)
        com = (com[0][:(z.size + 1), ...], com[1])
        load_com = False
    else:
        
        z = None
        load_com = True

    ihmm[t] = hdphmm.InfiniteHMM(com, traj_no=[t], load_com=load_com, difference=difference, 
                             observation_model=observation_model, order=order, max_states=max_states,
                             dim=dim, prior=prior, save_every=save_every, hyperparams=hyperparams,
                             seed_sequence=z)

In [ ]:
niter = 100
for i in traj_no:
    ihmm[i].inference(niter)
    
for i in traj_no:
    ihmm[i]._get_params(quiet=True)

In [ ]:
for t in traj_no:
    ihmm[t].summarize_results(savename='/home/ben/github/LLC_Membranes/Ben_Manuscripts/hdphmm/supporting_figures/many_states.pdf')

In [ ]:
ntrajper=10
endshow = 2000
MD_MSD = file_rw.load_object('trajectories/%s_msd.pl' % res)
time = np.arange(endshow) * 0.5
confidence = 68

unclustered_trajectory_realizations = np.zeros([len(ihmm), ihmm[traj_no[0]].z.shape[1], ntrajper, ihmm[traj_no[0]].dimensions])

for t in traj_no:
    
    final_p = get_params([ihmm[t]], ihmm[t], ihmm[t].z, clustered=False)
#     if mod_T:

#         final_p['T'] = modify_T(final_p['T'].mean(axis=0), percentile=20, increase_factor=2.25, decrease_factor=1)

    trajectory_generator = GenARData(params=final_p)

    for tr in tqdm.tqdm(range(ntrajper)):

        trajectory_generator.gen_trajectory(ihmm[t].z.shape[1], 1, bound_dimensions=[0, 1], resample_T=True, progress=False)

        unclustered_trajectory_realizations[i, :, tr, :] = trajectory_generator.traj[:, 0, :]

    plt.plot(time, MD_MSD.MSD[:endshow, t], color='black')

    msd = ts.msd(unclustered_trajectory_realizations[t, ...], 2, progress=False)
    msd_mean = msd.mean(axis=1)

    lower_confidence = (100 - confidence) / 2
    upper_confidence = 100 - lower_confidence

    error = np.zeros([2, msd.shape[0]], dtype=float)  # upper and lower bounds at each point along MSD curve
    # determine error bound for each tau (out of n MSD's, use that for the error bars)
    for s in range(error.shape[1]):
        error[0, s] = np.abs(np.percentile(msd[s, :], lower_confidence) - msd_mean[s])
        error[1, s] = np.abs(np.percentile(msd[s, :], upper_confidence) - msd_mean[s])

    plt.plot(time, msd.mean(axis=1)[:endshow], lw=2, color='xkcd:orange', label='IHMM')
    plt.fill_between(time, msd.mean(axis=1)[:endshow] + error[0, :endshow], msd.mean(axis=1)[:endshow] - error[1, :endshow], alpha=0.3, color='xkcd:orange')

    plt.show()

In [ ]:
for s in range(nseg):
    segments[s].summarize_results(traj_no=0)

In [ ]:
for i in traj_no:
    print('Trajectory %d' % i)
    ihmm[i].summarize_results(traj_no=0)

In [ ]:
mean_zero = []
dwells = []
hops = []

for s in range(nseg):
    
    zeroed, d, h = segments[s].subtract_mean(traj_no=0, simple_mean=True, return_dwells_hops=True)
    mean_zero.append(zeroed)
    dwells += d
    hops += h
    
hops = np.array(hops)[:, 1]
    
mean_zero = np.array(mean_zero)
mean_zero = mean_zero.reshape(mean_zero.shape[0] * mean_zero.shape[1], 2)
print(mean_zero.shape)

fig, ax = plt.subplots(2, 1, figsize=(12, 5), sharex=True)

ma_z = ts.calculate_moving_average(mean_zero[:, 1], 50)
ma_r = ts.calculate_moving_average(mean_zero[:, 0], 50)
print('z: %.3f' % ma_z.std())
print('r: %.3f' % ma_r.std())

ax[0].plot(ma_r)
ax[1].plot(ma_z)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from LLC_Membranes.llclib import timeseries as ts

mean_zero = []
dwells = []
hops = []

for t in traj_no:
    
    zeroed, d, h = ihmm[t].subtract_mean(traj_no=0, simple_mean=True, return_dwells_hops=True)
    mean_zero.append(zeroed)
    dwells += d
    hops += h
    
hops = np.array(hops)[:, 1]
    
mean_zero = np.array(mean_zero)

fig, ax = plt.subplots(2, 1, figsize=(12, 5), sharex=True)

ma_z = ts.calculate_moving_average(mean_zero[0, :, 1], 50)
ma_r = ts.calculate_moving_average(mean_zero[0, :, 0], 50)
print('z: %.3f' % ma_z.std())
print('r: %.3f' % ma_r.std())

ax[0].plot(ma_r)
ax[1].plot(ma_z)

plt.show()